# Asymptotic expansion of the energy {#sec-20230523130742}

This notebook uses $\LaTeX\newcommand{\D}{\mathrm{d}}\newcommand{\E}{\mathcal{E}}\newcommand{\order}[2][1]{#2^{(#1)}}\newcommand{\reals}{\mathbb{R}}$ custom macros.

In this chapter, we derive the asymptotic expansion (@eq-20230528190539) of the energy along the bifurcated branch. More precisely, the following quantity is expanded at the critical point

$$
\Delta\E(\eta) = \E[u(\eta), \lambda(\eta)] - \E[u^\ast \circ \lambda(\eta), \lambda(\eta)],
$$

where $\eta$ is an arbitrary parametrization of the bifurcated branch ($\eta = 0$ at the critical point).

The asymptotic expansion of the energy results from plugging the asymptotic expansions (@eq-20230528174310) and (@eq-20230411141319) of $\lambda(\eta)$ and $u(\eta)$ into the general expression of the energy that was derived in @sec-20230402140105.

In [2]:
from lsk import *
from sympy import *

We first define the asymptotic expansion of $\lambda$ w.r.t. the (unspecified) parameter $\eta$.

In [3]:
def create_λ():
    return λ1 * η + λ2 * η**2 / 2 + λ3 * η**3 / 6 + λ4 * η**4 / 24 + O(η**5)

In [4]:
#| code-fold: true
display_latex_equation(r"\lambda(\eta)", create_λ())

<IPython.core.display.Math object>

In the previous chapters, we have derived the asymptotic expansions of $\lambda$, $u^\ast$ and $u$ 
up to fourth-order in $\eta$. It will be shown below that these expansion lead to an expansion of $\Delta\E$ up to **fourth-order** terms in $\eta$.

For the sake of consistency, expansions of $\lambda$, $u^\ast$ and $u$ will be postulated up to fourth-order: it will be finally checked that the fourth-order expansion of $\Delta\E$ refers only to: $\order[1]{\lambda}$, $\order[2]{\lambda}$, $w_{i\lambda}$ and $w_{ij}$ (which are all known quantities).

The fundamental branch $u^\ast(\lambda)$ was previously defined as an asymptotic expansion w.r.t. $\lambda$. We can now define it as a function of $\eta$, keeping only fourth-order terms.

In [5]:
def create_u_star_2():
    return create_u_star(λ_fun=create_λ)

In [6]:
#| code-fold: true
display_latex_equation(r"u^\ast(\eta)", create_u_star_2().expand())

<IPython.core.display.Math object>

Similarly, we define the asymptotic expansion of $u$, see Eqs. (@eq-20230528174310) and (@eq-20230411141319); note that the quantities $\order[3]{\tilde{u}}$ and $\order[4]{\tilde{u}}$ are unknown (and will in fact disappear from the derivation). 

In [7]:
u1 = Symbol(r"{\order[1]{u}}")
u2_V = Symbol(r"{\order[2]{u}_V}")
u2_W = Symbol(r"{\order[2]{u}_W}")
u3 = Symbol(r"{\order[3]{u}}")
u4 = Symbol(r"{\order[4]{u}}")

def create_u():
    return (create_u_star(create_λ) + η * u1 + η**2 / 2 * (u2_V + u2_W) + η**3 / 6 * u3 + η**4 / 24 * u4)

In [8]:
#| code-fold: true
display_latex_equation(r"u(\eta)", create_u().expand())

<IPython.core.display.Math object>

We then use these expansions to evaluate the energy along the **fundamental** and **bifurcated** branches, as well as the difference $\Delta\E$ of these two quantities. The resulting expressions are too long to be displayed. We first apply some elementary simplifications.

In [9]:
E_fund = create_E(λ_fun=create_λ, u_fun=create_u_star_2).expand()
E_bif = create_E(λ_fun=create_λ, u_fun=create_u).expand()

In [10]:
ΔE = expand(E_bif - E_fund).subs({
    E2 * u1 : 0,
    E2 * u2_V : 0
})

In [11]:
assert ΔE.coeff(η, 0) == 0
assert ΔE.coeff(η, 1) == 0
assert ΔE.coeff(η, 2) == 0

In [26]:
d = {
    # Require first bifurcation equation
    E3 * u1 * u1 * u1 : -2 * λ1 * E2_dot * u1 * u1,
    E3 * u1 * u1 * u2_V : -2 * λ1 * E2_dot * u1 * u2_V,
    # Do not require bifurcation equations
    E2 * u2_W * u2_W : ((E4 * u1 * u1 * u1 * u1
                         - E_[i, j, k, l] * ξ1_[i] * ξ1_[j] * ξ1_[k] * ξ1_[l]) / 3
                        + 4 * λ1 / 3 * (E3_dot * u1 * u1 * u1
                                        - E_dot_[i, j, k] * ξ1_[i] * ξ1_[j] * ξ1_[k])
                        + 2 * λ1**2 * (E2_ddot * u1 * u1
                                       - E_ddot_[i, j] * ξ1_[i] * ξ1_[j])),
    E2_dot * u1 * u2_W : ((E_dot_[i, j, k] * ξ1_[i] * ξ1_[j] * ξ1_[k] 
                           - E3_dot * u1 * u1 * u1) / 3
                          + λ1 * (E_ddot_[i, j] * ξ1_[i] * ξ1_[j]
                                  - E2_ddot * u1 * u1)),
    E3 * u1 * u1 * u2_W  : ((E_[i, j, k, l] * ξ1_[i] * ξ1_[j] * ξ1_[k] * ξ1_[l]
                             - E4 * u1 * u1 * u1 * u1) / 3
                            + 2 * λ1 / 3 * (E_dot_[i, j, k] * ξ1_[i] * ξ1_[j] * ξ1_[k] 
                                            - E3_dot * u1 * u1 * u1)),
}

In [27]:
#| code-fold: true
display_latex_equation(r"\Delta\E", ΔE.subs(d).expand())

<IPython.core.display.Math object>

wich can be reordered as follows
$$
\Delta\E = \tfrac{1}{6} \eta^3 \, \Delta\order[3]{\E} + \tfrac{1}{24} \eta^4 \, \Delta\order[4]{\E},
$$
with
$$
\Delta\order[3]{\E} 
= \order[1]{\lambda} \, \dot{\E}_2(\order[1]{u}, \order[1]{u})
= \order[1]{\lambda} \, \dot{E}_{ij} \, \order[1]{\xi}_i \, \order[1]{\xi}_j
$$
and

$$
\begin{aligned}[b]
\order[4]{\Delta\E} ={} & E_{ijkl} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l + 4\order[1]{\lambda} \, \dot{E}_{ijk} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k\\
&+ 6 \bigl[ \bigl( \order[1]{\lambda} \bigr)^2 \, \ddot{E}_{ij} + \order[2]{\lambda} \, \dot{E}_{ij} \bigr] \, \order[1]{\xi}_i \, \order[1]{\xi}_j.
\end{aligned}
$$ {#eq-20230402101651}

and the asymptotic expansion (@eq-20230528190539) is retrieved.

## Asymptotic expansion of the hessian of the energy

We use the same code to derive an asymptotic expansion of the hessian $\E_{,uu}$ of the energy along the bifurcated branch.

In [ ]:
E_uu = (eval(data["E_uu"]).subs({Symbol("\lambda"): λ, Symbol("u"): u})  + O(η**3)).expand()

In [ ]:
#| code-fold: true
lsk.display_latex_equation(r"\E_{,uu}[u(\eta), \lambda(\eta)]", E_uu)

The above expression must be understood as follows

$$
\begin{aligned}[b]
\E_{,uu}[u(\eta), \lambda(\eta); \bullet, \bullet] ={} 
& \E_2(\bullet, \bullet) + \eta \, \bigl[ \order[1]{\xi}_i \, \E_3(v_i, \bullet, \bullet) + \order[1]{\lambda} \, \dot{\E}_2(\bullet, \bullet) \bigr]\\
&+ \tfrac{1}{2}\eta^2 \, \bigl[ \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_4(v_i, v_j, \bullet, \bullet)
+ \order[2]{\xi}_i \, \E_3(v_i, \bullet, \bullet) \\
&+ \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_3(w_{ij}, \bullet, \bullet)
+ 2\order[1]{\lambda} \, \order[1]{\xi}_i \, \E_3(w_{i\lambda}, \bullet, \bullet)\\
&+ 2\order[1]{\lambda} \, \order[1]{\xi}_i \, \dot{\E}_3(v_i, \bullet, \bullet)
+ (\order[1]{\lambda})^2 \, \ddot{\E}_2(\bullet, \bullet) + \order[2]{\lambda} \, \dot{\E}_2(\bullet, \bullet)\bigr],
\end{aligned}
$$

or, equivalently

$$
\begin{aligned}[b]
\E_{,uu}[u(\eta), \lambda(\eta); \bullet, \bullet] ={} 
& \E_2(\bullet, \bullet) + \eta \, \bigl[ \E_3(\order[1]{u}, \bullet, \bullet) + \order[1]{\lambda} \, \dot{\E}_2(\bullet, \bullet) \bigr]\\
&+ \tfrac{1}{2}\eta^2 \, \bigl[ \E_4(\order[1]{u}, \order[1]{u}, \bullet, \bullet)
+ \E_3(\order[2]{u}, \bullet, \bullet) \\
&+ 2\order[1]{\lambda} \, \dot{\E}_3(\order[1]{u}, \bullet, \bullet)
+ (\order[1]{\lambda})^2 \, \ddot{\E}_2(\bullet, \bullet) + \order[2]{\lambda} \, \dot{\E}_2(\bullet, \bullet)\bigr],
\end{aligned}
$$ {#eq-20230517061452}

where $\order[1]{u}$ and $\order[2]{u}$ are given by Eq. (@eq-20230411141319).